In [1]:
import subprocess
import os

In [2]:
!dx login --token  #[token]


Note: Use dx select --level VIEW or dx select --public to
select from projects for which you only have VIEW permissions.

Available projects (CONTRIBUTE or higher):
Setting current project to: Exome full cohort - Jan 05, 2023


In [3]:
outpath_dnanexus = '/Regenie_outfolder/TOPmed_LD'

os.system(f'dx mkdir {outpath_dnanexus}')

0

In [ ]:
%%bash

project="project-ID"
impute_file_dir='/TOPmed_downsample_5k'
data_field="ukb21007"
inpath_dnanexus='/Regenie_infolder/TOPmed_LD'
outpath_dnanexus='/Regenie_outfolder/TOPmed_LD'

# List of lead snps
csv_file="/Output/All_pheno_all_loci_topmed_list_38_unique.csv"

# Read CSV into an array, skipping the header
mapfile -t lines < <(tail -n +2 "$csv_file")

# Iterate over the lines using a for loop
for line in "${lines[@]}"; do
    IFS=, read -r Phenotype Locus_n Locus_name ID CHROM GENPOS ALLELE1 ALLELE0 A1FREQ MAF INFO BETA SE p <<< "$line"

    # Convert the colon-separated ID to underscore-separated
    file_lead_snp_id=$(echo $ID | tr ':' '_')
    lead_snp_id=$(echo $ID | tr '_' ':')
    
    # Extract the chromosome number from the SNP ID (format: chr_pos_allele1_allele2)
    chr=$(echo $CHROM)
    echo "Lead SNP ID: $lead_snp_id, Chromosome: $chr"

    # Construct the PLINK command
    run_plink_ld="plink --bfile ${data_field}_c${chr}_b0_v1_downsample_IDs \
        --r2 \
        --ld-snp ${lead_snp_id} \
        --keep-allele-order \
        --ld-window-kb 4000 \
        --ld-window-r2 0.1 \
        --ld-window 99999 \
        --out locus_${file_lead_snp_id}"

    # Run the PLINK command using dx toolkit
    dx run swiss-army-knife -iin="${impute_file_dir}/${data_field}_c${chr}_b0_v1_downsample_IDs.bed" \
     -iin="${impute_file_dir}/${data_field}_c${chr}_b0_v1_downsample_IDs.bim" \
     -iin="${impute_file_dir}/${data_field}_c${chr}_b0_v1_downsample_IDs.fam" \
     -icmd="${run_plink_ld}" --tag="TOPmed_ld_${file_lead_snp_id}" --instance-type "mem1_hdd1_v2_x16" \
     -imount_inputs=true \
     --destination="${project}:${outpath_dnanexus}" --verbose --yes
done
